# TODO list

* Overview of all available special modules like UART, SPI, etc.
* Improve report output

### 1. Prefix
This jupyter notebook can be used as a tool to create adapter board pin-mappings.
To be able to import the modules placed in the mt namespace, we need to add the directory containing the mt-module to the system path.

In [ ]:
import datetime as dt
import os
import sys
import pathlib as pl

PACKAGE_ROOT_PATH = str(pl.Path(os.getcwd()).parent.parent)
if PACKAGE_ROOT_PATH not in sys.path:
    sys.path.append(PACKAGE_ROOT_PATH)

from pinmap.board import Board
from pinmap.adapter import Adapter

### 2. Configuration and Setup
The tool requires two input files:
* The Baseboard pin definition, this file must contain following colums:
  * `Column`: Column of an EDB-Pin in the plug-grid, typically `A`, `B` or `C`
  * `Row`: Row number of an EDB-Pin in the plug-grid, typically `1`-`32`
  * `Bus`: Defines whether an EDB-Pin belongs to a bus, which requires the usage of the same MCU module, e.g. `UART0`.
  * `Signal`: The signal which is connected to the EDB-Pin, e.g. `UART0_TXD`.
  * `Status`: `Open` if the default jumper setting on the MT-EDB is open, `Closed` if it is closed.
  * `Regex-Module`: Regular expression to find all MCU modules which have pins that can be connected to this EDB-Pin, e.g. `FC`. Used to filter `ALT<x>_Module` inputs from `MCUBOARD_PINFILE_PATH`.
  * `Regex-Function`: Regular expression to find all MCU pins wich have the correct function to be connected to this EDB-Pin, e.g. `TX`. Used to filter `ALT<x>_Function` inputs from `MCUBOARD_PINFILE_PATH`.
* The MCU-Board pin definitions, this file must contain following colums:
  * `Board-Pin`: Identifier of the MCU-Pin on the MCU-Board, e.g. `J12_3`.
  * `MCU-Pin`: Identifier of the MCU-Pin at the MCU,  e.g. `PIO0_2`.
  * For a MCU that allows to route N different functions to a pin, N-pairs of the folloing columns:
    * `ALT<x>-Module`: Module connected if function `x` is selected, e.g. `FC0`. Leave empty if not available. See MCU-Documentation for details.
    * `ALT<x>-Function`: Function that is available at the MCU-Pin if function `x` is selected, e.g. `TXD_SCL_MISO`. Leave empty if not available. See MCU-Documentation for details.

**ATTENTION:**: The values in the Baseboard and MCU-Board pin definitions MUST NOT contain any hyphens (`-`), as this is used as separator for the mapping strings.

  For further configurations parameters please see the filebackend-Class constructor description.

In [ ]:
baseboard = Board(vendor="CME", longname="ExampleBaseboard", shortname="DMY", revision="A")
mcuboard  = Board(vendor="CME", longname="ExampleMcuBoard", shortname="SDMB", revision="A")
adapter   = Adapter(baseboard = baseboard, mcuboard=mcuboard, guiLabelIdWidth=70,
                    generate=(pl.Path(os.getcwd()).joinpath('example_baseboard.csv'), pl.Path(os.getcwd()).joinpath('example_mcuboard.csv'))
                   )

### 3. Mapping frontend
* The symbol `_\_` indicates that this signal jumper on the MT-EDB is open by default, i.e. this signal is not connected. This means, signal conflicts do not have an impact on the standard setting.
* All UARTs should be connected with high priority, as those are required for MT-RPC UART tests.
* If there are too few interface modules available or pin conflicts, one I2C bus could be operated with GPIO bitbanging mode instead of a dedicated I2C module. One I2C bus *must* be a dedicated I2C module though, to be able to test hardware I2C support.

In [ ]:
adapter.mappingFrontEnd

### 4. Export and re-import
To save the state of a mapping, call the export method of the mapping object:

In [ ]:
adapter.exportMapping()

In [ ]:
adapterReImported = Adapter(baseboard = baseboard, mcuboard=mcuboard, guiLabelIdWidth=70,
                            revision='A'
                           )

In [ ]:
adapterReImported.mappingFrontEnd